In [8]:
# imports
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import pprint
from riotwatcher import LolWatcher, ApiError
my_region = 'na1'
from api_key import API_KEY

lol_watcher = LolWatcher(API_KEY)

In [6]:
# select my riot puuid
my_region = 'na1'
me = lol_watcher.summoner.by_name(my_region, 'slayer56429')
puid = me["puuid"]


HTTPError: 403 Client Error: Forbidden for url: https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/slayer56429

In [ ]:
# find last 30 ranked games
match_hist = lol_watcher.match.matchlist_by_puuid(my_region,puid,count=30, queue = 420)

In [ ]:
# pretty print last game data
last = lol_watcher.match.by_id(my_region,"NA1_4493046493")

In [ ]:
# all participant data
participants = last["info"]["participants"]

part_num = 0
for p in participants:
    print(f"---------------------------------------Participant # {part_num}--------------------------------------------")
    part_num+=1
    pprint.pprint(p)

In [ ]:
# self data for loop
for p in participants:
    if puid in p['puuid']:
        self_game_data = p


In [ ]:
# remove challenges index
from itertools import islice
del self_game_data[next(islice(self_game_data, 7, None))]
pprint.pprint(self_game_data)

In [ ]:
#  remove perks key from dict
runes = self_game_data.pop('perks', None)

In [ ]:
# from rows to columns with .T
self_game_data = pd.DataFrame.from_dict(self_game_data,orient='index')
self_game_data.T

In [ ]:
# team data 
teams = last["info"]["teams"]
for t in teams:
    pprint.pprint(t)